In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings("ignore")

In [ ]:
ques_df = pd.read_csv('/content/drive/MyDrive/NLP/duplicate sent proj/ques_df_1.csv')
ques_df.head()

,question1,question2
0,what is the best java program to implement stack,what are some good java programs to practice with
1,can donald trump still win the election,does donald trump has chamces to win us elections
2,how can one be less of an introvert,can one be an extrovert but also an introvert
3,people should not have the right to commit sui...,why do not we have a right to commit suicide
4,how can i make money as a 13 year old,how can i make money as a thirteen year old


In [ ]:
final_df = pd.read_csv('/content/drive/MyDrive/NLP/duplicate sent proj/final_df_1.csv')
final_df.head()

,is_duplicate,q1_len,q2_len,q1_word,q2_word,word_common,total_words,word_share,cwc_min,cwc_max,...,ctc_max,last_word_equal,first_word_equal,abs_leng_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
0,0,48,49,9,9,3,18,0.166667,0.199996,0.249994,...,0.333330,0,1,1,9.0,0.265306,58,58,54,62
1,1,39,49,7,9,3,16,0.187500,0.599988,0.499992,...,0.333330,0,0,10,8.0,0.350000,68,64,73,73
2,0,35,45,8,9,5,16,0.312500,0.666644,0.499988,...,0.555549,1,0,10,8.5,0.361111,68,71,62,79
3,1,50,44,9,10,6,19,0.315789,0.749981,0.999967,...,0.599994,1,0,6,9.5,0.533333,74,84,70,84
4,0,37,43,10,10,9,20,0.450000,0.799984,0.799984,...,0.899991,1,1,6,10.0,0.684211,88,78,85,96


In [ ]:
pip install --upgrade pip setuptools wheel

In [ ]:
# pip install --upgrade gensim numpy scipy

In [ ]:
!pip install --upgrade numpy==1.24.3 scipy==1.10.1 gensim==4.3.1


In [ ]:
import os
import gensim
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize

# Tokenize each question
ques_df['q1_tokens'] = ques_df['question1'].apply(lambda x: word_tokenize(str(x).lower()))
ques_df['q2_tokens'] = ques_df['question2'].apply(lambda x: word_tokenize(str(x).lower()))

# Combine for training Word2Vec
questions = list(ques_df['q1_tokens']) + list(ques_df['q2_tokens'])

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=questions,
    vector_size=2000,
    window=5,
    min_count=2,
    workers=4
)

In [ ]:
def document_vector(tokens, model, vector_size):
    valid_tokens = [t for t in tokens if t in model.wv]
    return np.mean([model.wv[t] for t in valid_tokens], axis=0) if valid_tokens else np.zeros(vector_size)

vector_size = model.vector_size

q1_arr = np.vstack(ques_df['q1_tokens'].apply(lambda x: document_vector(x, model, vector_size)))
q2_arr = np.vstack(ques_df['q2_tokens'].apply(lambda x: document_vector(x, model, vector_size)))


In [ ]:

temp_df1 = pd.DataFrame(q1_arr, index = ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index = ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2],axis =1)

In [ ]:
final_df = pd.concat([final_df, temp_df], axis = 1)
final_df.shape


(30000, 4023)

In [ ]:
final_df.head()

,is_duplicate,q1_len,q2_len,q1_word,q2_word,word_common,total_words,word_share,cwc_min,cwc_max,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0,48,49,9,9,3,18,0.166667,0.199996,0.249994,...,0.139816,-0.111463,0.034502,-0.075046,0.053756,0.014013,-0.213570,-0.158463,0.005538,0.006554
1,1,39,49,7,9,3,16,0.187500,0.599988,0.499992,...,0.119491,-0.170474,0.153451,0.112216,0.050162,0.116545,-0.038905,0.025177,-0.031989,-0.111767
2,0,35,45,8,9,5,16,0.312500,0.666644,0.499988,...,0.047917,-0.125830,0.135296,0.062173,0.080340,0.037200,-0.086450,0.036057,-0.116073,-0.080420
3,1,50,44,9,10,6,19,0.315789,0.749981,0.999967,...,0.086976,-0.083773,0.115495,0.027990,0.066852,-0.038237,-0.177814,0.002259,-0.162670,0.021587
4,0,37,43,10,10,9,20,0.450000,0.799984,0.799984,...,0.227390,-0.048620,0.049992,0.015238,0.076978,0.177789,-0.153898,0.099052,-0.048115,0.008536


In [ ]:
x = final_df.iloc[:,1:].values
y = final_df.iloc[:,0].values

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
## Logistic regression
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nconfusion matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.7218333333333333

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.67      0.71      2973
           1       0.71      0.77      0.74      3027

    accuracy                           0.72      6000
   macro avg       0.72      0.72      0.72      6000
weighted avg       0.72      0.72      0.72      6000


confusion matrix:
 [[1998  975]
 [ 694 2333]]


In [ ]:
# Decision tree  classifier
model = DecisionTreeClassifier(criterion='gini', random_state=42)  # or use 'entropy'
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nconfusion matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.701

Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.70      0.70      2973
           1       0.70      0.70      0.70      3027

    accuracy                           0.70      6000
   macro avg       0.70      0.70      0.70      6000
weighted avg       0.70      0.70      0.70      6000


confusion matrix:
 [[2078  895]
 [ 899 2128]]


In [ ]:
# ranfom forest classifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nconfusion matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.783

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.72      0.77      2973
           1       0.76      0.84      0.80      3027

    accuracy                           0.78      6000
   macro avg       0.79      0.78      0.78      6000
weighted avg       0.79      0.78      0.78      6000


confusion matrix:
 [[2150  823]
 [ 479 2548]]


In [ ]:
# Xtreme gradient boost classifier
xgb = XGBClassifier()
xgb.fit(x_train, y_train)
y_pred = xgb.predict(x_test)
accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nconfusion matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.7811666666666667

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.74      0.77      2973
           1       0.76      0.83      0.79      3027

    accuracy                           0.78      6000
   macro avg       0.78      0.78      0.78      6000
weighted avg       0.78      0.78      0.78      6000


confusion matrix:
 [[2189  784]
 [ 529 2498]]
